## Using Javascript inside MLDB's Query Language statements with the built-in jeval function

MLDB provides and [SQL-like Query Language](../../../../doc/#builtin/sql/Sql.md.html) that contains many familiar SQL expressions. Further to this, MLDB also provides tailored [SQL value expressions and functions](../../../../doc/builtin/sql/ValueExpression.md.html) for your queries. For example, we have created a suite of [Built-in Functions](../../../../doc/builtin/sql/ValueExpression.md.html#builtinfunctions). This includes [jeval](../../../../doc/#builtin/sql/ValueExpression.md.html), for evaluating a JS function from SQL.

Let's start the tutorial. The notebook cells below use `pymldb`'s `Connection` class to make [REST API](../../../../doc/#builtin/WorkingWithRest.md.html) calls. You can check out the [Using `pymldb` Tutorial](../../../../doc/nblink.html#_tutorials/Using pymldb Tutorial) for more details.

In [1]:
from pymldb import Connection
mldb = Connection("http://localhost")

In the [loading data from HTTP server or online file storage services](../../../../ipy/notebooks/_tutorials/_latest/Loading%20data%20from%20HTTP%20servers%20or%20online%20file%20storage%20services.ipynb#) tutorial, we loaded a specific file from an archive that was located on the [Stanford Network Analysis Project (SNAP)](http://snap.stanford.edu/index.html) website. The dataset contains all other users that are within user no. 3980's set of circles. We will structure the data below using jeval. Let's check's out the unformated version of the data first.

In [44]:
dataUrl = "http://snap.stanford.edu/data/facebook.tar.gz"

mldb.put("/v1/procedures/import_data", {
    "type": "import.text",
    "params": {
        "dataFileUrl": "archive+" + dataUrl + "#facebook/3980.circles",
        "delimiter": " ", 
        "quotechar": "",
        "outputDataset": "import_URL2",
        "runOnCreation": True
    }
})

mldb.query("SELECT * NAMED rowName() FROM import_URL2 LIMIT 10")

,"""circle0 3989 4009"""
_rowName,
2,circle1\t4010\t4037
3,circle2\t4013
4,circle3\t4024\t3987\t4015
5,circle4\t4006
6,circle5\t4035
7,circle6\t3999\t4028\t4005\t3996\t4031\t4018\t3...
8,circle7\t3984
9,circle8\t3988\t4030\t4026\t4021
10,circle9\t3983\t3992\t4033\t4017\t4000\t3986


Ideally, we would like to know the 'user ids' that are within each of user 3980's set of circles. As you can see though, the data has no column name. Furthermore, if we seperate each tab delimited value, we will end up with an unknown number of columns. The code below uses the built-in [jeval function](../../../../doc/builtin/sql/ValueExpression.md.html#builtinfunctions) and outputs the data into a [sparse.mutable](../../../../doc/#builtin/datasets/MutableSparseMatrixDataset.md.html)  type dataset to solve the aforementioned roadblocks.

In [2]:
dataUrl = "http://snap.stanford.edu/data/facebook.tar.gz"

print mldb.put("/v1/procedures/import_non_formated", {
    "type": "import.text",
    "params": {
        "dataFileUrl": "archive+" + dataUrl + "#facebook/3980.circles",
        "headers": ["circles"],
        "select": """
            jseval('
                var row_val = val.split("\t");
                var rtn = {};
                rtn["rowName"] = row_val[0];
                for(i=1; i<row_val.length; i++) {
                    rtn[row_val[i]] = 1;
                }
                return rtn;
                ','val', circles) AS *
        """,
        "outputDataset": {
            "id": "import_non_formated",
            "type": "sparse.mutable"
        },
        "runOnCreation": True
    }
})

<Response [201]>


In [3]:
mldb.query("""
    SELECT * EXCLUDING(rowName)
    NAMED rowName
    FROM import_non_formated 
    ORDER BY CAST(rowName() AS integer) 
    LIMIT 5
""")

,3989,4009,4010,4037,4013,3987,4015,4024,4006
_rowName,,,,,,,,,
circle0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
circle1,NaN,NaN,1,1,NaN,NaN,NaN,NaN,NaN
circle2,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
circle3,NaN,NaN,NaN,NaN,NaN,1,1,1,NaN
circle4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Structuring this dataset would have taken a few steps with just SQL. Here using jeval and sparse mutable type dataset, we can organize our data in a single step and without knowing all of the characteristics of the dataset.

MLDB has the capability to organize unstructured data in a seamless way. This adds greater flexibility when dealing with different types of input data.

## Where to next?

Check out the other [Tutorials and Demos](../../../../doc/#builtin/Demos.md.html).